In [ ]:
# 영상처리 과제3: 마우스로 로고 스탬프 찍기
# 소프트웨어학과 2020301081 허종우
# 배경 이미지에 마우스로 클릭하면 opencv logo 이미지가 클릭한 위치에 스탬프로 찍듯이 출력되도록 하시오.
# 다른 위치에 클릭하면 로고 이미지가 이동되도록 하되, 로고 이미지 전체가 나오지 않는 위치에 클릭하면 출력하지 않도록 하시오.

In [34]:
import numpy as np
import cv2

# 배경 이미지와 로고 이미지 읽기
test = cv2.imread("C:/Users/82107/Desktop/source/images/test.jpg", cv2.IMREAD_COLOR) # 배경
logo = cv2.imread("C:/Users/82107/Desktop/source/images/logo.jpg", cv2.IMREAD_COLOR) # 로고

if test is None or logo is None: raise Exception("영상파일 읽기 오류")

test_copy = test.copy() # 범위 외 클릭 시 기존 이미지 출력을 위한 이미지 복사

# 로고 영상 이진화
masks = cv2.threshold(logo, 220, 255, cv2.THRESH_BINARY)[1]
masks = cv2.split(masks)

# 전경 마스크와 배경 마스크 생성
fg_pass_mask = cv2.bitwise_or(masks[0], masks[1])
fg_pass_mask = cv2.bitwise_or(masks[2], fg_pass_mask)
bg_pass_mask = cv2.bitwise_not(fg_pass_mask)

(test_h, test_w) = test.shape[:2]
(logo_h, logo_w) = logo.shape[:2]  # 로고 영상 크기

def onMouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:           # 마우스 왼쪽 버튼을 누르는 이벤트
        global test, logo, fg_pass_mask, bg_pass_mask
        
        # 배경을 원본으로 초기화 (이전 로고 지우기)
        test[:] = test_copy

        x = x - logo_w // 2            # 마우스 클릭 부분이 로고 이미지의 중앙
        y = y - logo_h // 2
        
        # 로고가 배경 안에 들어가는지 확인
        if x >= 0 and x + logo_w <= test_w and y>= 0 and y + logo_h <= test_h:
            # 클릭한 위치에 로고 이미지 복사
            roi = test[y:y + logo_h, x:x + logo_w]                      # 관심 영역(roi) 지정
            
            foreground = cv2.bitwise_and(logo, logo, mask=fg_pass_mask) # 로고의 전경만 복사
            background = cv2.bitwise_and(roi, roi, mask=bg_pass_mask)   # 원본 roi의 배경만 복사
        
            dst = cv2.add(background, foreground)                       # 배경과 전경을 합성
            test[y:y + logo_h, x:x + logo_w] = dst                      # 합성된 이미지를 배경에 적용
            
            # 결과 이미지를 갱신하여 보여줌
            cv2.imshow('dst', test)
        else:                        # 로고가 배경안에 들어가지 않으면 
            test[:] = test_copy      # 스탬프가 지워진 원본 배경으로 초기화
            cv2.imshow('dst', test)

# 초기 이미지 출력
cv2.imshow('dst', test)

# 마우스 콜백 함수 등록
cv2.namedWindow('dst')
cv2.setMouseCallback('dst', onMouse)

cv2.waitKey(0)
cv2.destroyAllWindows()